In [1]:
import os
import json
import numpy as np
import pandas as pd
import scipy.sparse as spr

목차

- 1. 데이터 처리 (불러오기,추가,수정,변환)
- 2. 플레이리스트로 추천 (협업필터링)

# 데이터 처리

## train data 불러오기

In [2]:
with open('data/train.json',encoding='utf-8-sig') as f:
    train_dict = json.load(f)
    
with open('data/song_meta.json',encoding='utf-8-sig') as f:
    song_dict = json.load(f)
    
with open('data/genre_gn_all.json',encoding='utf-8-sig') as f:
    genre_dict = json.load(f)
    
train_df = pd.DataFrame.from_dict(train_dict)
song_df = pd.DataFrame.from_dict(song_dict)

## train_df 전처리

### 일반 전처리
- num_songs,num_tags = '중복제거한 노래 태그 개수'

In [3]:
train_df.head(3)

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000


In [4]:
song_df.head(3)

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2


In [5]:
train_df['tags_cnt'] = train_df['tags'].map(lambda x : len(x))
train_df['songs_cnt'] = train_df['songs'].map(lambda x : len(x))

In [6]:
def list_merger(x):
    temp_list.extend(x)

temp_list = []

train_df['songs'].map(list_merger)

unique_songs = list(set(temp_list))

num_songs = len(unique_songs)

In [7]:
temp_list = []
                
train_df['tags'].map(list_merger)
                    
unique_tags = list(set(temp_list))
                
num_tags = len(unique_tags)

In [8]:
num_songs

615142

In [9]:
num_tags

29160

In [10]:
num_train = len(train_df)
num_train

115071

### tag에 id부여

In [11]:
train_df.head(3)

,tags,id,plylst_title,songs,like_cnt,updt_date,tags_cnt,songs_cnt
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,1,19
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000,2,42
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000,2,28


In [12]:
# { 태그 : 새로운id } 딕셔너리
tag_to_id = {}


# { 새로운id : 태그 } 딕셔너리
id_to_tag = {}

# enumerate 반복 횟수를 이용해서 새로운 id 부여
for i,v in enumerate(unique_tags):
    id_to_tag[i] = v

# id_to_tag의 키 밸류를 반전해서 tag_to_id를 만듦
tag_to_id = {v:k for k,v in id_to_tag.items()}


# tags를 id로 변환해서 새로운 column을 생성
def tag_id_trans(x):
    
    temp_list = []
    
    for tag in x:
        temp_list.append(tag_to_id[tag])
    
    return temp_list

train_df['tags_id'] = train_df['tags'].map(tag_id_trans)

In [13]:
train_df.head()

,tags,id,plylst_title,songs,like_cnt,updt_date,tags_cnt,songs_cnt,tags_id
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,1,19,[16811]
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000,2,42,"[28930, 14240]"
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000,2,28,"[16947, 24595]"
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000,10,38,"[18518, 758, 5057, 28540, 25739, 749, 25569, 2..."
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000,1,53,[12312]


### 플레이리스트 속 노래들(songs)에게 새로운 id 부여

In [14]:
# { 노래id : 새로운id } 딕셔너리
song_to_id = {}


# { 새로운id : 노래id } 딕셔너리
id_to_song = {}

# enumerate 반복 횟수를 이용해서 새로운 id 부여
for i,v in enumerate(unique_songs):
    id_to_song[i] = v

# id_to_tag의 키 밸류를 반전해서 tag_to_id를 만듦
song_to_id = {v:k for k,v in id_to_song.items()}


# tags를 id로 변환해서 새로운 column을 생성
def song_id_trans(x):
    
    temp_list = []
    
    for song in x:
        temp_list.append(song_to_id[song])
    
    return temp_list

train_df['new_songs_id'] = train_df['songs'].map(song_id_trans)

In [15]:
train_df.head(3)

,tags,id,plylst_title,songs,like_cnt,updt_date,tags_cnt,songs_cnt,tags_id,new_songs_id
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,1,19,[16811],"[456704, 112732, 333158, 488440, 258853, 12127..."
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000,2,42,"[28930, 14240]","[375894, 587314, 431997, 104605, 338568, 21226..."
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000,2,28,"[16947, 24595]","[72132, 240434, 144495, 161861, 307991, 222934..."


### 플레이리스트 새로운 id 부여

In [16]:
train_df['new_plyst_id'] = train_df.index

In [17]:
id_to_playlist = {}

playlist_to_id = {}

def mk_dict(x):
    
    id_to_playlist[x['new_plyst_id']] = x['id']

train_df.loc[:,['id','new_plyst_id']].apply(mk_dict,axis=1)

playlist_to_id = {v:k for k,v in id_to_playlist.items()}

In [18]:
train_df = train_df[['tags','tags_id','id','new_plyst_id','plylst_title','songs','new_songs_id','tags_cnt','songs_cnt','like_cnt','updt_date']]
train_df.head(3)

,tags,tags_id,id,new_plyst_id,plylst_title,songs,new_songs_id,tags_cnt,songs_cnt,like_cnt,updt_date
0,[락],[16811],61281,0,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...","[456704, 112732, 333158, 488440, 258853, 12127...",1,19,71,2013-12-19 18:36:19.000
1,"[추억, 회상]","[28930, 14240]",10532,1,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...","[375894, 587314, 431997, 104605, 338568, 21226...",2,42,1,2014-12-02 16:19:42.000
2,"[까페, 잔잔한]","[16947, 24595]",76951,2,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...","[72132, 240434, 144495, 161861, 307991, 222934...",2,28,17,2017-08-28 07:09:34.000


In [19]:
train_df.columns = ['태그','태그_id','플리_id','새플리_id','플리제목','노래_id','새노래_id','태그수','노래수','좋아요수','갱신일']
train_df.head(3)

,태그,태그_id,플리_id,새플리_id,플리제목,노래_id,새노래_id,태그수,노래수,좋아요수,갱신일
0,[락],[16811],61281,0,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...","[456704, 112732, 333158, 488440, 258853, 12127...",1,19,71,2013-12-19 18:36:19.000
1,"[추억, 회상]","[28930, 14240]",10532,1,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...","[375894, 587314, 431997, 104605, 338568, 21226...",2,42,1,2014-12-02 16:19:42.000
2,"[까페, 잔잔한]","[16947, 24595]",76951,2,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...","[72132, 240434, 144495, 161861, 307991, 222934...",2,28,17,2017-08-28 07:09:34.000


### 변수 정리

<현재>

- num_songs : 중복 제거된 총 노래수 615142
- num_tags : 중복 제거된 총 태그수 29160
- num_train : 데이터의 총 갯수 115071
 
- tag_to_id : { 태그 : 새로운id } 딕셔너리
- id_to_tag : { 새로운id : 태그 } 딕셔너리
- song_to_id : { 노래 : 새로운id } 딕셔너리
- id_to_song : { 새로운id : 노래 } 딕셔너리
- playlist_to_id { 플리id : 새로운id } 딕셔너리 
- id_to_playlist { 새로운id : 플리id } 딕셔너리 

<이후>
- code_to_genre {장르코드: 장르} 딕셔너리
- genre_to_code {장르 : 장르코드} 딕셔너리
- gcode_to_id  {장르코드 : 장르_id} 딕셔너리
- id_to_gcode  {장르_id : 장르코드} 딕셔너리

# 장르와 코드로 협업필터링 추천해보자

## genre 앞의 네글자로 대장르만 가져오기

In [20]:
genre_big = {}

# 모든 장르 딕셔너리를 돌면서
for k,v in genre_dict.items():
    
    # 맨 뒤 두자리가 00이면 대장류로 분류
    if k[-2:] == '00':
        
        # 맨앞 네자리를 키로 하는 대장류 딕셔너리 값 추가
        genre_big[k[:4]] = v

# 찍어보기
genre_big

{'GN01': '발라드',
 'GN02': '댄스',
 'GN03': '랩/힙합',
 'GN04': 'R&B/Soul',
 'GN05': '인디음악',
 'GN06': '록/메탈',
 'GN07': '성인가요',
 'GN08': '포크/블루스',
 'GN09': 'POP',
 'GN10': '록/메탈',
 'GN11': '일렉트로니카',
 'GN12': '랩/힙합',
 'GN13': 'R&B/Soul',
 'GN14': '포크/블루스/컨트리',
 'GN15': 'OST',
 'GN16': '클래식',
 'GN17': '재즈',
 'GN18': '뉴에이지',
 'GN19': 'J-POP',
 'GN20': '월드뮤직',
 'GN21': 'CCM',
 'GN22': '어린이/태교',
 'GN23': '종교음악',
 'GN24': '국악',
 'GN25': '아이돌',
 'GN26': '일렉트로니카',
 'GN27': 'EDM',
 'GN28': '뮤직테라피',
 'GN29': '뮤지컬',
 'GN30': '크리스마스'}

## 세부장르에 대분류 장르의 이름 넣기

In [21]:
code_to_genre = {}

# 모든 딕셔너리를 돌면서
for k,v in genre_dict.items():
    
    # 맨뒤 두자리가 00이 아니면 대장류가 아닌거임!
    if k[-2:] != '00':
        
        # 그럴떈 아까만든 대장르 딕셔너리의 대장류 이름을 추가해서 이름을 수정해서 다시 넣어줌
        new_value = genre_big[k[:4]]+'_'+v
        code_to_genre[k] = new_value
        
    # 아니면 그대로
    else:
        code_to_genre[k] = v

# 찍어보기
code_to_genre

{'GN0100': '발라드',
 'GN0101': '발라드_세부장르전체',
 'GN0102': "발라드_'80",
 'GN0103': "발라드_'90",
 'GN0104': "발라드_'00",
 'GN0105': "발라드_'10-",
 'GN0200': '댄스',
 'GN0201': '댄스_세부장르전체',
 'GN0202': "댄스_'80",
 'GN0203': "댄스_'90",
 'GN0204': "댄스_'00",
 'GN0205': "댄스_'10-",
 'GN0300': '랩/힙합',
 'GN0301': '랩/힙합_세부장르전체',
 'GN0302': '랩/힙합_랩 스타일',
 'GN0303': '랩/힙합_보컬 스타일',
 'GN0304': '랩/힙합_언더그라운드 힙합',
 'GN0305': '랩/힙합_시대별',
 'GN0400': 'R&B/Soul',
 'GN0401': 'R&B/Soul_세부장르전체',
 'GN0402': 'R&B/Soul_어반',
 'GN0403': 'R&B/Soul_R&B',
 'GN0500': '인디음악',
 'GN0501': '인디음악_세부장르전체',
 'GN0502': '인디음악_포크',
 'GN0503': '인디음악_록',
 'GN0504': '인디음악_일렉',
 'GN0505': '인디음악_힙합',
 'GN0506': '인디음악_발라드',
 'GN0507': "인디음악_'90",
 'GN0508': "인디음악_'00",
 'GN0509': "인디음악_'10-",
 'GN0600': '록/메탈',
 'GN0601': '록/메탈_세부장르전체',
 'GN0602': "록/메탈_'70",
 'GN0603': "록/메탈_'80",
 'GN0604': "록/메탈_'90",
 'GN0605': "록/메탈_'00",
 'GN0606': "록/메탈_'10-",
 'GN0700': '성인가요',
 'GN0701': '성인가요_세부장르전체',
 'GN0702': '성인가요_신세대트로트',
 'GN0703': '성인가요_전설의트로트',
 'GN0

## 장르 딕셔너리 키 밸류 바꾸기 -> 나중에 장르 이름으로 코드 찾을 거기 때문임!

In [22]:
genre_to_code = {v:k for k,v in code_to_genre.items()}
# 찍어보기
genre_to_code 

{'발라드': 'GN0100',
 '발라드_세부장르전체': 'GN0101',
 "발라드_'80": 'GN0102',
 "발라드_'90": 'GN0103',
 "발라드_'00": 'GN0104',
 "발라드_'10-": 'GN0105',
 '댄스': 'GN0200',
 '댄스_세부장르전체': 'GN0201',
 "댄스_'80": 'GN0202',
 "댄스_'90": 'GN0203',
 "댄스_'00": 'GN0204',
 "댄스_'10-": 'GN0205',
 '랩/힙합': 'GN1200',
 '랩/힙합_세부장르전체': 'GN1201',
 '랩/힙합_랩 스타일': 'GN0302',
 '랩/힙합_보컬 스타일': 'GN0303',
 '랩/힙합_언더그라운드 힙합': 'GN0304',
 '랩/힙합_시대별': 'GN0305',
 'R&B/Soul': 'GN1300',
 'R&B/Soul_세부장르전체': 'GN1301',
 'R&B/Soul_어반': 'GN1304',
 'R&B/Soul_R&B': 'GN0403',
 '인디음악': 'GN0500',
 '인디음악_세부장르전체': 'GN0501',
 '인디음악_포크': 'GN0502',
 '인디음악_록': 'GN0503',
 '인디음악_일렉': 'GN0504',
 '인디음악_힙합': 'GN0505',
 '인디음악_발라드': 'GN0506',
 "인디음악_'90": 'GN0507',
 "인디음악_'00": 'GN0508',
 "인디음악_'10-": 'GN0509',
 '록/메탈': 'GN1000',
 '록/메탈_세부장르전체': 'GN1001',
 "록/메탈_'70": 'GN1010',
 "록/메탈_'80": 'GN1011',
 "록/메탈_'90": 'GN1012',
 "록/메탈_'00": 'GN1013',
 "록/메탈_'10-": 'GN1014',
 '성인가요': 'GN0700',
 '성인가요_세부장르전체': 'GN0701',
 '성인가요_신세대트로트': 'GN0702',
 '성인가요_전설의트로트': 'GN0703',
 '성인가

## Train과 장르 합치기
- 플레이리스트 안에 들어있는 노래의 장르를 중복제거 해서 컬럼에 추가
- 35초 정도 걸렸음

In [23]:
def code_gen(x):
    
    # 모든 장르를 담을 리스트
    temp_list = []
       
    # 플레이리스트에 들어있는 장르를 extend로 싸그리 모아줌
    # song노래의 id는 index와 같다는 점을 참고!
    for id in x:
        temp_list.extend(song_df.loc[id,'song_gn_dtl_gnr_basket'])
        
    # 중복제거
    temp_list = list(set(temp_list))
    
    return temp_list

train_df['장르'] = train_df['노래_id'].map(code_gen)
train_df.head(3)

,태그,태그_id,플리_id,새플리_id,플리제목,노래_id,새노래_id,태그수,노래수,좋아요수,갱신일,장르
0,[락],[16811],61281,0,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...","[456704, 112732, 333158, 488440, 258853, 12127...",1,19,71,2013-12-19 18:36:19.000,"[GN0904, GN1912, GN1102, GN1901, GN1402, GN110..."
1,"[추억, 회상]","[28930, 14240]",10532,1,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...","[375894, 587314, 431997, 104605, 338568, 21226...",2,42,1,2014-12-02 16:19:42.000,"[GN1803, GN2603, GN1708, GN0901, GN2602, GN170..."
2,"[까페, 잔잔한]","[16947, 24595]",76951,2,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...","[72132, 240434, 144495, 161861, 307991, 222934...",2,28,17,2017-08-28 07:09:34.000,"[GN2603, GN0401, GN1703, GN0402, GN1701, GN080..."


## 새로운 장르 코드 생성

In [24]:
gcode_unique = []
gcode_to_id = {}
id_to_gcode = {}

def g_unique(x):
    global gcode_unique
    
    gcode_unique.extend(x)

train_df['장르'].map(g_unique)    

# 장르 중복제거 해서 정렬
gcode_unique = sorted(list(set(gcode_unique)))

In [25]:
# id : gcode 딕셔너리 생성
for i,v in enumerate(gcode_unique):
    id_to_gcode[i] = v
    
id_to_gcode

{0: 'GN0101',
 1: 'GN0102',
 2: 'GN0103',
 3: 'GN0104',
 4: 'GN0105',
 5: 'GN0201',
 6: 'GN0202',
 7: 'GN0203',
 8: 'GN0204',
 9: 'GN0205',
 10: 'GN0301',
 11: 'GN0302',
 12: 'GN0303',
 13: 'GN0304',
 14: 'GN0401',
 15: 'GN0402',
 16: 'GN0403',
 17: 'GN0501',
 18: 'GN0502',
 19: 'GN0503',
 20: 'GN0504',
 21: 'GN0505',
 22: 'GN0506',
 23: 'GN0507',
 24: 'GN0508',
 25: 'GN0509',
 26: 'GN0601',
 27: 'GN0602',
 28: 'GN0603',
 29: 'GN0604',
 30: 'GN0605',
 31: 'GN0606',
 32: 'GN0701',
 33: 'GN0702',
 34: 'GN0703',
 35: 'GN0704',
 36: 'GN0705',
 37: 'GN0706',
 38: 'GN0707',
 39: 'GN0708',
 40: 'GN0709',
 41: 'GN0710',
 42: 'GN0801',
 43: 'GN0802',
 44: 'GN0803',
 45: 'GN0804',
 46: 'GN0805',
 47: 'GN0901',
 48: 'GN0902',
 49: 'GN0903',
 50: 'GN0904',
 51: 'GN0905',
 52: 'GN0906',
 53: 'GN0907',
 54: 'GN0908',
 55: 'GN1001',
 56: 'GN1002',
 57: 'GN1003',
 58: 'GN1004',
 59: 'GN1005',
 60: 'GN1006',
 61: 'GN1007',
 62: 'GN1008',
 63: 'GN1009',
 64: 'GN1010',
 65: 'GN1011',
 66: 'GN1012',
 67: 

In [26]:
gcode_to_id = {v:k for k,v in id_to_gcode.items()}

gcode_to_id

{'GN0101': 0,
 'GN0102': 1,
 'GN0103': 2,
 'GN0104': 3,
 'GN0105': 4,
 'GN0201': 5,
 'GN0202': 6,
 'GN0203': 7,
 'GN0204': 8,
 'GN0205': 9,
 'GN0301': 10,
 'GN0302': 11,
 'GN0303': 12,
 'GN0304': 13,
 'GN0401': 14,
 'GN0402': 15,
 'GN0403': 16,
 'GN0501': 17,
 'GN0502': 18,
 'GN0503': 19,
 'GN0504': 20,
 'GN0505': 21,
 'GN0506': 22,
 'GN0507': 23,
 'GN0508': 24,
 'GN0509': 25,
 'GN0601': 26,
 'GN0602': 27,
 'GN0603': 28,
 'GN0604': 29,
 'GN0605': 30,
 'GN0606': 31,
 'GN0701': 32,
 'GN0702': 33,
 'GN0703': 34,
 'GN0704': 35,
 'GN0705': 36,
 'GN0706': 37,
 'GN0707': 38,
 'GN0708': 39,
 'GN0709': 40,
 'GN0710': 41,
 'GN0801': 42,
 'GN0802': 43,
 'GN0803': 44,
 'GN0804': 45,
 'GN0805': 46,
 'GN0901': 47,
 'GN0902': 48,
 'GN0903': 49,
 'GN0904': 50,
 'GN0905': 51,
 'GN0906': 52,
 'GN0907': 53,
 'GN0908': 54,
 'GN1001': 55,
 'GN1002': 56,
 'GN1003': 57,
 'GN1004': 58,
 'GN1005': 59,
 'GN1006': 60,
 'GN1007': 61,
 'GN1008': 62,
 'GN1009': 63,
 'GN1010': 64,
 'GN1011': 65,
 'GN1012': 66,
 'GN1

In [27]:
# 장르_id 칼럼을 생성

def c_to_id(x):
    
    temp_list = []
    
    for code in x:
        temp_list.append(gcode_to_id[code])
        
    return temp_list


train_df['장르_id'] = train_df['장르'].map(c_to_id)

train_df.head(3)

,태그,태그_id,플리_id,새플리_id,플리제목,노래_id,새노래_id,태그수,노래수,좋아요수,갱신일,장르,장르_id
0,[락],[16811],61281,0,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...","[456704, 112732, 333158, 488440, 258853, 12127...",1,19,71,2013-12-19 18:36:19.000,"[GN0904, GN1912, GN1102, GN1901, GN1402, GN110...","[50, 153, 70, 142, 96, 69, 48, 66, 47, 59, 145..."
1,"[추억, 회상]","[28930, 14240]",10532,1,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...","[375894, 587314, 431997, 104605, 338568, 21226...",2,42,1,2014-12-02 16:19:42.000,"[GN1803, GN2603, GN1708, GN0901, GN2602, GN170...","[137, 205, 133, 47, 204, 126, 3, 46, 18, 102, ..."
2,"[까페, 잔잔한]","[16947, 24595]",76951,2,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...","[72132, 240434, 144495, 161861, 307991, 222934...",2,28,17,2017-08-28 07:09:34.000,"[GN2603, GN0401, GN1703, GN0402, GN1701, GN080...","[205, 14, 128, 15, 126, 46, 18, 26, 0, 20, 16,..."


## 태그_id 하나당 어떤 장르가 포함되었는지 araboja

- 각 태그가 몇번 출몰 했는지 알아야 함

### 태그 출몰 횟수 구하기

In [28]:
# 중복을 제거한 tag 리스트

tag_all = []

def collect_tag(x):
    global tag_all
    
    tag_all.extend(x)
    
train_df['태그_id'].map(collect_tag)

tag_unique = list(set(tag_all))

In [29]:
# 각 태그가 몇번 출몰 했는지 세어 놓은 딕셔너리

from collections import Counter

tag_count = dict(Counter(tag_all))

tag_count = {i:tag_count[i] for i in range(len(tag_count))}

tag_count = sorted(tag_count.items(),key=(lambda x:x[1]),reverse = True)
tag_count = {k:v for k,v in tag_count[:5]}
tag_count

In [30]:
sum(list(tag_count.values()))

476331

In [31]:
# 태그_id에 달린 장르id들을 넣을 딕셔너리 초기화
tid_gid_dict = {i:[] for i in range(len(tag_unique))}

In [32]:
len(tag_unique)

29160

In [33]:
train_df.head(3)

,태그,태그_id,플리_id,새플리_id,플리제목,노래_id,새노래_id,태그수,노래수,좋아요수,갱신일,장르,장르_id
0,[락],[16811],61281,0,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...","[456704, 112732, 333158, 488440, 258853, 12127...",1,19,71,2013-12-19 18:36:19.000,"[GN0904, GN1912, GN1102, GN1901, GN1402, GN110...","[50, 153, 70, 142, 96, 69, 48, 66, 47, 59, 145..."
1,"[추억, 회상]","[28930, 14240]",10532,1,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...","[375894, 587314, 431997, 104605, 338568, 21226...",2,42,1,2014-12-02 16:19:42.000,"[GN1803, GN2603, GN1708, GN0901, GN2602, GN170...","[137, 205, 133, 47, 204, 126, 3, 46, 18, 102, ..."
2,"[까페, 잔잔한]","[16947, 24595]",76951,2,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...","[72132, 240434, 144495, 161861, 307991, 222934...",2,28,17,2017-08-28 07:09:34.000,"[GN2603, GN0401, GN1703, GN0402, GN1701, GN080...","[205, 14, 128, 15, 126, 46, 18, 26, 0, 20, 16,..."


In [34]:
# 태그_id에 달린 장르id들을 list로 넣어주기

counter = 0

def tagid_genre(x):
    
    global counter
    global tid_gid_dict
    temp_list = []
    for i in x['태그_id']:
        tid_gid_dict[i].extend(x['장르_id'])

train_df[['태그_id','장르_id']].apply(tagid_genre,axis=1)

0         None
1         None
2         None
3         None
4         None
          ... 
115066    None
115067    None
115068    None
115069    None
115070    None
Length: 115071, dtype: object

In [35]:
len(tid_gid_dict)

29160

In [36]:
for i in range(len(tid_gid_dict)):
    v_count = Counter(tid_gid_dict[i])
    v_count_sorted = sorted(v_count.items(),key=(lambda x:x[1]),reverse=True)

    temp_list = []
    
    if len(v_count_sorted) > 4:
        for k,v in v_count_sorted[:5]:
            temp_list.append((k,v))
    else:
        for k,v in v_count_sorted:
            temp_list.append((k,v))
            
    tid_gid_dict[i] = temp_list

    
tid_gid_dict

{0: [(79, 1), (78, 1), (94, 1), (90, 1), (88, 1)],
 1: [(104, 1), (48, 1), (47, 1), (89, 1), (102, 1)],
 2: [(143, 1), (142, 1), (153, 1)],
 3: [(12, 1), (197, 1), (10, 1), (8, 1), (198, 1)],
 4: [(69, 1), (70, 1), (79, 1), (104, 1), (86, 1)],
 5: [(135, 2), (140, 2), (126, 2), (141, 1), (127, 1)],
 6: [(143, 1), (142, 1), (205, 1), (14, 1), (15, 1)],
 7: [(14, 2), (5, 2), (3, 2), (46, 2), (18, 2)],
 8: [(47, 5), (131, 5), (126, 5), (48, 4), (31, 4)],
 9: [(60, 1), (55, 1)],
 10: [(32, 1), (1, 1), (43, 1), (45, 1), (40, 1)],
 11: [(144, 1), (142, 1), (96, 1), (13, 1), (18, 1)],
 12: [(137, 1), (110, 1), (140, 1), (128, 1), (126, 1)],
 13: [(209, 1), (14, 1), (47, 1), (15, 1), (23, 1)],
 14: [(17, 2), (31, 2), (26, 2), (25, 2), (19, 2)],
 15: [(14, 49), (47, 48), (0, 47), (17, 45), (25, 45)],
 16: [(17, 16), (31, 16), (26, 16), (25, 16), (19, 16)],
 17: [(46, 3), (18, 3), (24, 3), (31, 3), (17, 3)],
 18: [(47, 1), (6, 1), (5, 1), (81, 1), (3, 1)],
 19: [(144, 1), (143, 1), (142, 1), (17

In [37]:
# 포함된 장르의 개수를 가지고 있는 list
genre_len = [len(v) for v in tid_gid_dict.values()]

In [38]:
#  태그별 장르포함 횟수만큼 반복  * 2번 태그는 7번 나왔으니 2를 7번 찍기
row = np.repeat(range(len(tid_gid_dict)),genre_len)
len(row)

142936

In [39]:
# 장르를 하나씩 뽑아서 넣어주기
col =  [genre[0] for k,v in tid_gid_dict.items() for genre in v]
len(col)

142936

In [40]:
tid_gid_dict

{0: [(79, 1), (78, 1), (94, 1), (90, 1), (88, 1)],
 1: [(104, 1), (48, 1), (47, 1), (89, 1), (102, 1)],
 2: [(143, 1), (142, 1), (153, 1)],
 3: [(12, 1), (197, 1), (10, 1), (8, 1), (198, 1)],
 4: [(69, 1), (70, 1), (79, 1), (104, 1), (86, 1)],
 5: [(135, 2), (140, 2), (126, 2), (141, 1), (127, 1)],
 6: [(143, 1), (142, 1), (205, 1), (14, 1), (15, 1)],
 7: [(14, 2), (5, 2), (3, 2), (46, 2), (18, 2)],
 8: [(47, 5), (131, 5), (126, 5), (48, 4), (31, 4)],
 9: [(60, 1), (55, 1)],
 10: [(32, 1), (1, 1), (43, 1), (45, 1), (40, 1)],
 11: [(144, 1), (142, 1), (96, 1), (13, 1), (18, 1)],
 12: [(137, 1), (110, 1), (140, 1), (128, 1), (126, 1)],
 13: [(209, 1), (14, 1), (47, 1), (15, 1), (23, 1)],
 14: [(17, 2), (31, 2), (26, 2), (25, 2), (19, 2)],
 15: [(14, 49), (47, 48), (0, 47), (17, 45), (25, 45)],
 16: [(17, 16), (31, 16), (26, 16), (25, 16), (19, 16)],
 17: [(46, 3), (18, 3), (24, 3), (31, 3), (17, 3)],
 18: [(47, 1), (6, 1), (5, 1), (81, 1), (3, 1)],
 19: [(144, 1), (143, 1), (142, 1), (17

In [41]:
# 태그 마다 달린 장르의 총길이 만큼 1찍기
dat = np.repeat(1, sum(genre_len))
len(dat)

142936

In [42]:
train_tags_A = spr.csr_matrix((dat, (row, col)), shape=(train_df['태그수'].sum(), len(gcode_to_id)))

train_tags_A_T = train_tags_A.T.tocsr()

In [43]:
train_tags_A.toarray().reshape(train_df['태그수'].sum(), len(gcode_to_id))

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

# 내가 고른 장르와 연관된 장르를 뽑자@!@

## 나의 장르 리스트 만들기

In [44]:
def recom_genres(tag):
    #my_genres = ['인디음악_발라드']

    #my_genre_code = [genre_to_code[g] for g in my_genres]

    #my_genre_ids = [gcode_to_id[c] for c in my_genre_code]

    my_genre_ids = tid_gid_dict[tag_to_id[tag]]
    
    my_genre_ids
    
    # 내가 뽑은 장르 id 만 1인 1차원 희소행렬

    p = np.zeros((len(gcode_to_id),1))

    for id,count in my_genre_ids:
        p[id]=1
        
    # 희소행렬과 dot 연산을 통해 각 태그와 겹치는 장르들의 갯수를 추출
    val = train_tags_A.dot(p).reshape(-1)
    
    cand_gen = train_tags_A_T.dot(val)
    
    # 1차원으로 바꿔서 중복된 장르를 가진 태그 상위 150개 도출
    cand_gen_idx = cand_gen.reshape(-1).argsort()[-150:][::-1]

    # 비슷한 장르 10개 도출
    cand_gen_idx = cand_gen_idx[:10]

    cand_gen_idx
    
    # 원래 노래 id값으로 복원
    result_ids = [id_to_gcode[i] for i in cand_gen_idx]
    
    
    result_genres = [(code_to_genre[i]) for i in result_ids]
    
    return result_genres

In [45]:
recom_genres('지오디')

['아이돌_세부장르전체',
 '아이돌_발라드',
 '아이돌_남자 아이돌',
 "발라드_'00",
 'R&B/Soul_세부장르전체',
 '댄스_세부장르전체',
 "발라드_'10-",
 '랩/힙합_보컬 스타일',
 "포크/블루스_'10-",
 "댄스_'00"]